# 🔍 혐오와 갈등은 만들어지는가?
## 텍스트 마이닝을 활용한 온라인 담론 분석 교육

---

### 📚 학습 목표
1. 온라인 텍스트 데이터 수집 및 전처리 방법 이해
2. 감성 분석을 통한 혐오 표현 탐지
3. 토픽 모델링으로 갈등 주제 파악
4. 시각화를 통한 패턴 발견

---

## Part 1: 환경 설정 및 라이브러리 설치

In [ ]:
# 필요한 라이브러리 설치
!pip install konlpy pandas numpy matplotlib seaborn wordcloud scikit-learn
!pip install transformers torch
!pip install pyLDAvis gensim

# KoNLPy를 위한 Java 설정 (Colab)
!apt-get update
!apt-get install -y openjdk-11-jdk
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'

In [ ]:
# 기본 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (Colab)
!apt-get install -y fonts-nanum
!fc-cache -fv

plt.rcParams['font.family'] = 'NanumBarunGothic'
plt.rcParams['axes.unicode_minus'] = False

print("✅ 환경 설정 완료!")

## Part 2: 샘플 데이터 생성

실제 교육에서는 웹 크롤링으로 데이터를 수집하지만, 여기서는 교육용 샘플 데이터를 사용합니다.

⚠️ **윤리적 고려사항**: 실제 데이터 수집 시 개인정보 보호와 저작권을 반드시 준수해야 합니다.

In [ ]:
# 교육용 샘플 댓글 데이터 생성
# 실제 분석에서는 뉴스 API, YouTube API 등을 활용

sample_comments = [
    # 중립적 의견
    {"text": "이 문제는 다양한 관점에서 볼 필요가 있습니다", "source": "뉴스댓글", "topic": "정책"},
    {"text": "객관적인 데이터를 보고 판단해야 할 것 같아요", "source": "뉴스댓글", "topic": "정책"},
    {"text": "양쪽 의견 모두 일리가 있네요", "source": "유튜브", "topic": "사회"},
    {"text": "더 많은 정보가 필요한 것 같습니다", "source": "뉴스댓글", "topic": "경제"},
    {"text": "전문가 의견을 들어봐야겠어요", "source": "유튜브", "topic": "정책"},
    
    # 건설적 비판
    {"text": "이 정책은 재검토가 필요해 보입니다", "source": "뉴스댓글", "topic": "정책"},
    {"text": "논리적으로 몇 가지 문제점이 있어요", "source": "뉴스댓글", "topic": "사회"},
    {"text": "대안을 함께 제시해주시면 좋겠습니다", "source": "유튜브", "topic": "경제"},
    {"text": "이 부분은 개선이 필요합니다", "source": "커뮤니티", "topic": "정책"},
    {"text": "근거를 좀 더 명확히 해주세요", "source": "뉴스댓글", "topic": "사회"},
    
    # 부정적/공격적 표현 (분석 대상)
    {"text": "이건 완전 말도 안 되는 소리야", "source": "유튜브", "topic": "정책"},
    {"text": "도대체 뭘 하는 건지 모르겠다", "source": "뉴스댓글", "topic": "정책"},
    {"text": "이런 기사 왜 쓰는지 이해가 안 됨", "source": "뉴스댓글", "topic": "언론"},
    {"text": "기자가 편파적이네요", "source": "뉴스댓글", "topic": "언론"},
    {"text": "한쪽 얘기만 하고 있잖아", "source": "유튜브", "topic": "언론"},
    
    # 갈등 유발성 표현
    {"text": "그쪽 사람들은 항상 이런 식이야", "source": "커뮤니티", "topic": "사회"},
    {"text": "역시 예상대로네", "source": "유튜브", "topic": "정책"},
    {"text": "이게 나라냐", "source": "뉴스댓글", "topic": "정책"},
    {"text": "국민을 뭘로 보는 거야", "source": "뉴스댓글", "topic": "정책"},
    {"text": "진짜 답이 없다", "source": "커뮤니티", "topic": "사회"},
    
    # 긍정적 표현
    {"text": "좋은 정보 감사합니다", "source": "유튜브", "topic": "정보"},
    {"text": "이런 분석 기사 좋아요", "source": "뉴스댓글", "topic": "언론"},
    {"text": "응원합니다 힘내세요", "source": "유튜브", "topic": "사회"},
    {"text": "좋은 방향으로 가고 있네요", "source": "뉴스댓글", "topic": "정책"},
    {"text": "기대됩니다", "source": "커뮤니티", "topic": "경제"},
]

# DataFrame 생성
df = pd.DataFrame(sample_comments)
df['id'] = range(1, len(df) + 1)
df['date'] = pd.date_range(start='2024-01-01', periods=len(df), freq='D')

print(f"📊 총 {len(df)}개의 샘플 댓글 데이터 생성")
df.head(10)

## Part 3: 텍스트 전처리

In [ ]:
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

def preprocess_text(text):
    """텍스트 전처리 함수"""
    # 특수문자 제거 (한글, 영문, 숫자만 유지)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    # 연속 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def extract_keywords(text):
    """명사 키워드 추출"""
    nouns = okt.nouns(text)
    # 한 글자 명사 제외
    return [noun for noun in nouns if len(noun) > 1]

def extract_morphs(text):
    """형태소 분석 (명사, 형용사, 동사)"""
    morphs = okt.pos(text)
    return [word for word, pos in morphs if pos in ['Noun', 'Adjective', 'Verb'] and len(word) > 1]

# 전처리 적용
df['cleaned_text'] = df['text'].apply(preprocess_text)
df['keywords'] = df['cleaned_text'].apply(extract_keywords)
df['morphs'] = df['cleaned_text'].apply(extract_morphs)

print("✅ 텍스트 전처리 완료!")
df[['text', 'cleaned_text', 'keywords']].head()

## Part 4: 감성 분석 (Sentiment Analysis)

### 4.1 규칙 기반 감성 분석

In [ ]:
# 감성 사전 정의 (교육용 간소화 버전)
positive_words = [
    '좋다', '좋아요', '좋은', '감사', '응원', '기대', '훌륭', '최고', 
    '긍정', '희망', '발전', '개선', '협력', '이해', '존중', '공감'
]

negative_words = [
    '싫다', '나쁘다', '문제', '걱정', '불만', '실망', '답답', '화나',
    '말도 안', '이해 안', '모르겠', '답이 없', '한심', '편파'
]

conflict_words = [
    '그쪽', '너희', '걔네', '항상', '역시', '뻔하', '그러니까', 
    '이게 나라', '국민을', '이런 식'
]

def analyze_sentiment_rule(text):
    """규칙 기반 감성 분석"""
    pos_count = sum(1 for word in positive_words if word in text)
    neg_count = sum(1 for word in negative_words if word in text)
    conflict_count = sum(1 for word in conflict_words if word in text)
    
    # 점수 계산
    score = pos_count - neg_count - (conflict_count * 1.5)
    
    if score > 0:
        return 'positive', score
    elif score < 0:
        return 'negative', score
    else:
        return 'neutral', score

def detect_conflict_potential(text):
    """갈등 유발 가능성 탐지"""
    conflict_count = sum(1 for word in conflict_words if word in text)
    if conflict_count >= 2:
        return 'high'
    elif conflict_count == 1:
        return 'medium'
    else:
        return 'low'

# 분석 적용
df['sentiment_result'] = df['text'].apply(analyze_sentiment_rule)
df['sentiment'] = df['sentiment_result'].apply(lambda x: x[0])
df['sentiment_score'] = df['sentiment_result'].apply(lambda x: x[1])
df['conflict_potential'] = df['text'].apply(detect_conflict_potential)

print("✅ 감성 분석 완료!")
df[['text', 'sentiment', 'sentiment_score', 'conflict_potential']].head(10)

### 4.2 딥러닝 기반 감성 분석 (KoBERT 활용)

In [ ]:
# Hugging Face의 한국어 감성 분석 모델 활용
from transformers import pipeline

try:
    # 한국어 감성 분석 파이프라인
    sentiment_pipeline = pipeline(
        "sentiment-analysis",
        model="snunlp/KR-FinBert-SC",  # 한국어 감성 분석 모델
        tokenizer="snunlp/KR-FinBert-SC"
    )
    
    def analyze_sentiment_dl(text):
        """딥러닝 기반 감성 분석"""
        try:
            result = sentiment_pipeline(text[:512])[0]  # 최대 512 토큰
            return result['label'], result['score']
        except:
            return 'unknown', 0.0
    
    # 일부 샘플에 적용 (시간 소요로 인해)
    sample_texts = df['text'].head(5).tolist()
    for text in sample_texts:
        label, score = analyze_sentiment_dl(text)
        print(f"텍스트: {text[:30]}... → 감성: {label} (신뢰도: {score:.2f})")
        
except Exception as e:
    print(f"⚠️ 딥러닝 모델 로드 실패: {e}")
    print("규칙 기반 분석 결과를 사용합니다.")

## Part 5: 시각화 및 패턴 분석

In [ ]:
# 시각화 설정
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. 감성 분포
sentiment_counts = df['sentiment'].value_counts()
colors = {'positive': '#4CAF50', 'neutral': '#FFC107', 'negative': '#F44336'}
axes[0, 0].pie(sentiment_counts.values, labels=sentiment_counts.index, 
               autopct='%1.1f%%', colors=[colors.get(s, '#999') for s in sentiment_counts.index])
axes[0, 0].set_title('📊 전체 감성 분포', fontsize=14, fontweight='bold')

# 2. 출처별 감성 분포
source_sentiment = pd.crosstab(df['source'], df['sentiment'], normalize='index') * 100
source_sentiment.plot(kind='bar', ax=axes[0, 1], color=['#F44336', '#FFC107', '#4CAF50'])
axes[0, 1].set_title('📱 출처별 감성 분포', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('비율 (%)')
axes[0, 1].set_xticklabels(axes[0, 1].get_xticklabels(), rotation=45)
axes[0, 1].legend(title='감성')

# 3. 갈등 유발 가능성 분포
conflict_counts = df['conflict_potential'].value_counts()
conflict_colors = {'low': '#4CAF50', 'medium': '#FFC107', 'high': '#F44336'}
axes[1, 0].bar(conflict_counts.index, conflict_counts.values, 
               color=[conflict_colors.get(c, '#999') for c in conflict_counts.index])
axes[1, 0].set_title('⚠️ 갈등 유발 가능성 분포', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('댓글 수')

# 4. 주제별 부정적 감성 비율
topic_negative = df[df['sentiment'] == 'negative'].groupby('topic').size()
topic_total = df.groupby('topic').size()
negative_ratio = (topic_negative / topic_total * 100).sort_values(ascending=True)
negative_ratio.plot(kind='barh', ax=axes[1, 1], color='#F44336')
axes[1, 1].set_title('📈 주제별 부정적 감성 비율', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('부정적 감성 비율 (%)')

plt.tight_layout()
plt.savefig('sentiment_analysis_result.png', dpi=150, bbox_inches='tight')
plt.show()

print("📊 시각화 저장 완료: sentiment_analysis_result.png")

## Part 6: 워드클라우드 생성

In [ ]:
from wordcloud import WordCloud

# 전체 키워드 수집
all_keywords = []
for keywords in df['keywords']:
    all_keywords.extend(keywords)

# 빈도수 계산
keyword_freq = Counter(all_keywords)

# 감성별 키워드 분리
positive_keywords = []
negative_keywords = []

for _, row in df.iterrows():
    if row['sentiment'] == 'positive':
        positive_keywords.extend(row['keywords'])
    elif row['sentiment'] == 'negative':
        negative_keywords.extend(row['keywords'])

# 워드클라우드 생성 함수
def create_wordcloud(word_freq, title, colormap='viridis'):
    # 나눔고딕 폰트 경로 (Colab)
    font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
    
    wc = WordCloud(
        font_path=font_path,
        width=800,
        height=400,
        background_color='white',
        colormap=colormap,
        max_words=100
    ).generate_from_frequencies(dict(word_freq))
    
    return wc

# 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 전체 워드클라우드
wc_all = create_wordcloud(keyword_freq, '전체', 'viridis')
axes[0].imshow(wc_all, interpolation='bilinear')
axes[0].set_title('🔤 전체 키워드', fontsize=14, fontweight='bold')
axes[0].axis('off')

# 긍정 워드클라우드
if positive_keywords:
    wc_pos = create_wordcloud(Counter(positive_keywords), '긍정', 'Greens')
    axes[1].imshow(wc_pos, interpolation='bilinear')
axes[1].set_title('😊 긍정적 댓글 키워드', fontsize=14, fontweight='bold')
axes[1].axis('off')

# 부정 워드클라우드
if negative_keywords:
    wc_neg = create_wordcloud(Counter(negative_keywords), '부정', 'Reds')
    axes[2].imshow(wc_neg, interpolation='bilinear')
axes[2].set_title('😠 부정적 댓글 키워드', fontsize=14, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.savefig('wordcloud_result.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 상위 10개 키워드:")
for word, count in keyword_freq.most_common(10):
    print(f"  {word}: {count}회")

## Part 7: 토픽 모델링 (LDA)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 형태소 분석된 텍스트 준비
df['morphs_text'] = df['morphs'].apply(lambda x: ' '.join(x))

# CountVectorizer로 문서-단어 행렬 생성
vectorizer = CountVectorizer(max_features=1000, min_df=1)
doc_term_matrix = vectorizer.fit_transform(df['morphs_text'])

# LDA 모델 학습
n_topics = 4
lda_model = LatentDirichletAllocation(
    n_components=n_topics,
    random_state=42,
    max_iter=10
)
lda_model.fit(doc_term_matrix)

# 토픽별 주요 단어 출력
feature_names = vectorizer.get_feature_names_out()

print("\n📑 토픽 모델링 결과")
print("=" * 50)

topic_labels = ['정책 비판', '언론 신뢰', '사회 갈등', '긍정적 반응']

for topic_idx, topic in enumerate(lda_model.components_):
    top_words_idx = topic.argsort()[-8:][::-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(f"\n토픽 {topic_idx + 1} ({topic_labels[topic_idx]}):")
    print(f"  주요 키워드: {', '.join(top_words)}")

# 각 문서의 주요 토픽 할당
doc_topics = lda_model.transform(doc_term_matrix)
df['main_topic'] = doc_topics.argmax(axis=1) + 1
df['topic_confidence'] = doc_topics.max(axis=1)

print("\n" + "=" * 50)
print("✅ 토픽 모델링 완료!")

## Part 8: 혐오 표현 패턴 분석

In [ ]:
# 혐오 표현 패턴 사전 (교육용 - 실제 연구에서는 더 정교한 사전 사용)
hate_patterns = {
    '일반화': ['항상', '걔네', '그쪽', '너희', '다들', '모든'],
    '비하': ['한심', '멍청', '무식', '저질'],
    '선동': ['각성', '분노', '일어나', '심판'],
    '편가르기': ['~충', '~놈', '~년', '좌파', '우파', '진보', '보수']
}

def detect_hate_patterns(text):
    """혐오 표현 패턴 탐지"""
    detected = []
    for pattern_type, keywords in hate_patterns.items():
        for keyword in keywords:
            if keyword in text:
                detected.append((pattern_type, keyword))
    return detected

# 패턴 탐지 적용
df['hate_patterns'] = df['text'].apply(detect_hate_patterns)
df['has_hate_pattern'] = df['hate_patterns'].apply(lambda x: len(x) > 0)

# 결과 분석
print("🔍 혐오 표현 패턴 분석 결과")
print("=" * 50)
print(f"\n전체 댓글 수: {len(df)}")
print(f"혐오 패턴 포함 댓글: {df['has_hate_pattern'].sum()}개 ({df['has_hate_pattern'].mean()*100:.1f}%)")

# 패턴 유형별 빈도
pattern_counts = {}
for patterns in df['hate_patterns']:
    for pattern_type, _ in patterns:
        pattern_counts[pattern_type] = pattern_counts.get(pattern_type, 0) + 1

print("\n패턴 유형별 빈도:")
for pattern_type, count in sorted(pattern_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {pattern_type}: {count}회")

# 혐오 패턴 포함 댓글 예시
print("\n혐오 패턴 포함 댓글 예시:")
hate_comments = df[df['has_hate_pattern']]
for _, row in hate_comments.head(3).iterrows():
    print(f"  - \"{row['text']}\"")
    print(f"    탐지된 패턴: {row['hate_patterns']}")

## Part 9: 종합 분석 리포트 생성

In [ ]:
# 종합 분석 리포트
print("\n" + "=" * 60)
print("📊 텍스트 마이닝 종합 분석 리포트")
print("=" * 60)

print("\n1️⃣ 데이터 개요")
print(f"   - 분석 대상: {len(df)}개 댓글")
print(f"   - 출처 분포: {dict(df['source'].value_counts())}")
print(f"   - 주제 분포: {dict(df['topic'].value_counts())}")

print("\n2️⃣ 감성 분석 결과")
sentiment_dist = df['sentiment'].value_counts(normalize=True) * 100
for sentiment, pct in sentiment_dist.items():
    print(f"   - {sentiment}: {pct:.1f}%")

print("\n3️⃣ 갈등 유발 가능성")
conflict_dist = df['conflict_potential'].value_counts(normalize=True) * 100
for level, pct in conflict_dist.items():
    print(f"   - {level}: {pct:.1f}%")

print("\n4️⃣ 주요 발견사항")
print(f"   - 가장 부정적인 주제: {df[df['sentiment']=='negative']['topic'].mode().values[0] if len(df[df['sentiment']=='negative']) > 0 else 'N/A'}")
print(f"   - 가장 갈등 유발 가능성 높은 출처: {df[df['conflict_potential']=='high']['source'].mode().values[0] if len(df[df['conflict_potential']=='high']) > 0 else 'N/A'}")
print(f"   - 혐오 패턴 탐지율: {df['has_hate_pattern'].mean()*100:.1f}%")

print("\n5️⃣ 교육적 시사점")
print("   - 온라인 댓글에서 갈등은 특정 패턴과 표현을 통해 형성/확산됨")
print("   - 일반화, 비하, 편가르기 표현이 갈등 증폭에 기여")
print("   - 텍스트 마이닝으로 이러한 패턴을 객관적으로 분석 가능")
print("   - 미디어 리터러시 교육에 데이터 기반 접근 필요")

print("\n" + "=" * 60)

## Part 10: 결과 저장 및 내보내기

In [ ]:
# 분석 결과 CSV 저장
export_df = df[['id', 'text', 'source', 'topic', 'sentiment', 'sentiment_score', 
                'conflict_potential', 'main_topic', 'has_hate_pattern']]
export_df.to_csv('analysis_result.csv', index=False, encoding='utf-8-sig')

# 요약 통계 저장
summary_stats = {
    'total_comments': len(df),
    'positive_ratio': (df['sentiment'] == 'positive').mean(),
    'negative_ratio': (df['sentiment'] == 'negative').mean(),
    'high_conflict_ratio': (df['conflict_potential'] == 'high').mean(),
    'hate_pattern_ratio': df['has_hate_pattern'].mean()
}

import json
with open('summary_stats.json', 'w', encoding='utf-8') as f:
    json.dump(summary_stats, f, ensure_ascii=False, indent=2)

print("✅ 결과 파일 저장 완료:")
print("   - analysis_result.csv")
print("   - summary_stats.json")
print("   - sentiment_analysis_result.png")
print("   - wordcloud_result.png")

---

## 📚 추가 학습 자료

### 실제 데이터 수집 방법

1. **네이버 뉴스 댓글**: 네이버 API 또는 BeautifulSoup
2. **유튜브 댓글**: YouTube Data API v3
3. **트위터/X**: Twitter API v2

### 주의사항
- 개인정보 보호법 준수
- 저작권 및 이용약관 확인
- IRB(연구윤리위원회) 승인 (학술 연구 시)

### 심화 학습
- BERT 기반 혐오 표현 분류기 학습
- 시계열 분석을 통한 갈등 확산 패턴
- 네트워크 분석을 통한 영향력 분석

---

## 🎯 토론 질문

1. 온라인 혐오 표현을 기술적으로 탐지하는 것의 한계는?
2. 맥락 없이 단어만으로 혐오를 판단할 수 있는가?
3. 플랫폼의 알고리즘이 갈등 증폭에 기여하는가?
4. 텍스트 마이닝 결과를 정책에 어떻게 활용할 수 있는가?